#ROMAN URDU TEXT RECOGNITION  

####Text extraction using OCR, followed by a module that recognizes whether the text is in Roman Urdu or not

##Group Members:

####Muneeza Iftikhar (02-136212-012)

####Hafsa Hafeez Siddiqui (02-136212-026)


## Installing libraries and mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 29.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)


## Live/Webcam Detection

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

## Preprocessing

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import os
from PIL import Image, ImageEnhance, ImageFilter

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])

  # Create a folder to save the images
  folder_path = '/content/captured_images'
  if not os.path.exists(folder_path):
    os.makedirs(folder_path)

  # Save the captured image in the folder
  with open(os.path.join(folder_path, filename), 'wb') as f:
    f.write(binary)

  return os.path.join(folder_path, filename)

def apply_preprocessing(filename):
    # Open the image file.
    with Image.open(filename) as img:
        # Convert the image to greyscale.
        grey_img = img.convert('L')

        # Sharpen the image.
        sharp_img = grey_img.filter(ImageFilter.SHARPEN)

        # Resize the image.
        resized_img = sharp_img.resize((400, 300))

        # Save the pre-processed image.
        resized_img.save(os.path.join(os.path.dirname(filename), 'preprocessed_' + os.path.basename(filename)))

try:
    filename = take_photo()
    print('Saved to {}'.format(filename))

    # Show the image which was just taken.
    display(Image.open(filename))

    # Apply pre-processing on the captured image.
    apply_preprocessing(filename)
    print('Pre-processed image saved as preprocessed_' + os.path.basename(filename))

    # Show the pre-processed image.
    display(Image.open(os.path.join(os.path.dirname(filename), 'preprocessed_' + os.path.basename(filename))))
except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))

## Text Extraction

In [ ]:
from PIL import Image, ImageFilter
import easyocr
import os

# Assuming the captured image is saved as 'photo.jpg' in the '/content/captured_images' folder
image_path = '/content/captured_images/photo.jpg'

def extract_text_easy_ocr(image_path):
    # Perform OCR
    reader = easyocr.Reader(['en'])
    result = reader.readtext(image_path)
    text = ' '.join([line[1] for line in result])

    # Return the extracted text
    return text

def run_easy_ocr(image_path):
    extracted_text = extract_text_easy_ocr(image_path)
    print(f'Extracted Text: ' + extracted_text)

# Run OCR on the captured image
run_easy_ocr(image_path)

## Preprocessing of Urdu Dictonary

In [ ]:
file_path = '/content/drive/MyDrive/Roman-Urdu-Dictionary.txt'

In [ ]:
import re
import string

def preprocess_dictionary(file_path):
    # Step 1: Load the dictionary
    with open(file_path, 'r', encoding='utf-8') as f:
        roman_urdu_dict = f.readlines()

    # Step 2: Clean the data
    cleaned_dict = []
    for entry in roman_urdu_dict:
        entry = entry.strip()  # Remove leading and trailing whitespace
        entry = entry.lower()  # Convert to lowercase
        # Tokenization based on the first colon only
        parts = entry.split(':', 1)
        if len(parts) == 2:
            word, meanings = parts
            word = word.strip()
            # Remove punctuation from the word
            word = word.translate(str.maketrans('', '', string.punctuation))
            # Split meanings on slashes and commas
            meanings_list = [meaning.strip() for meaning in re.split(r'[/,]', meanings)]
            for meaning in meanings_list:
                cleaned_dict.append((word, meaning))

    # Step 3: Remove duplicate entries (if any)
    unique_entries = set(cleaned_dict)

    # Step 4: Organize the data
    organized_dict = {}
    for word, meaning in unique_entries:
        if word in organized_dict:
            organized_dict[word].add(meaning)
        else:
            organized_dict[word] = {meaning}

    # Convert sets to lists for easier usage later
    for word in organized_dict:
        organized_dict[word] = list(organized_dict[word])

    return organized_dict

# Example usage:
preprocessed_dict = preprocess_dictionary(file_path)

# Now, preprocessed_dict contains the cleaned and organized Roman Urdu dictionary
# You can use this dictionary in your OCR task for identifying and categorizing Roman Urdu words.

# import pprint  # Pretty print to visualize the dictionary
# pprint.pprint(preprocessed_dict)


## Roman Urdu Module

In [ ]:
import nltk
nltk.download('words')

In [ ]:
import glob
import os
from PIL import Image
import nltk
from nltk.corpus import words

with open(file_path, 'r') as file:
    roman_urdu_words_set = set(line.strip() for line in file)

def run_easy_ocr(image_path):
    # """
    # Run OCR on an image.
    # Args:
    #     image_path (str): The path to the image to run OCR on.
    # Returns:
    #     str: The extracted text.
    # """
    english_words = set(words.words())  # Load English dictionary
    all_roman_urdu_words = []  # List to store all Roman Urdu words
    all_english_words = []  # List to store all English words

    extracted_text = extract_text_easy_ocr(image_path)

    words_in_text = extracted_text.split()  # Split the text into words
    roman_urdu_words = []
    english_words_in_text = []

    for word in words_in_text:
        if word.lower() in roman_urdu_words_set:  # Check if the word is in your Roman Urdu dictionary
            roman_urdu_words.append(word)  # It's likely Roman Urdu
        elif word.lower() not in english_words:  # Check if the word is not in the English dictionary
            roman_urdu_words.append(word)  # It's likely Roman Urdu
        else:
            english_words_in_text.append(word)  # It's an English word

    print(f'Extracted Text: {extracted_text}')
    print(f'Roman Urdu Words: {roman_urdu_words}')  # Print the Roman Urdu words
    print(f'English Words: {english_words_in_text}')  # Print the English words

    all_roman_urdu_words.extend(roman_urdu_words)  # Add Roman Urdu words to the list
    all_english_words.extend(english_words_in_text)  # Add English words to the list

    # print("All Roman Urdu Words:", all_roman_urdu_words)  # Print the list of all Roman Urdu words
    # print("All English Words:", all_english_words)  # Print the list of all English words

run_easy_ocr(image_path)